## Demonstration of ogcapi API

The initial attempt to use the `owslib` library failed due to issus in the requests getting through to the F5.

Work from here down uses `requests` directly.

In [1]:
import folium
import requests

### Get some GeoJSON data and display it directly on a `folium` map.

In [39]:
# Get the data
params = {'f': 'json', 'limit': 10}
response = requests.get('https://ogcapi.bgs.ac.uk/collections/recentearthquakes/items', params=params)
data = response.json()

In [75]:
# Define an initial map centred on and zoomed to the UK
m = folium.Map(location=[55, -2], zoom_start=5)

In [76]:
# Use a style function to set the style dynamically per data point
# In this case use the magnitude to set the radius of the marker
def style_function(feature):
    radius = feature['properties']['ml'] * 5
    return {'radius': radius,
            'color': 'red',
            'weight': 2}

In [77]:
# Create a basic unstyled tooltip
tooltip = folium.features.GeoJsonTooltip(
    fields=['ml'],
    aliases=['magnitude'])

In [78]:
# Create a basic unstyled popup
popup = folium.features.GeoJsonPopup(
    fields=["year", "latitude", "longitude", "depth", "ml", "intensity"],
    aliases=["year", "latitude", "longitude", "depth", "magnitude", "intensity"])

In [79]:
# Populate and display the map
folium.GeoJson(
    data,
    tooltip=tooltip,
    popup=popup,
    style_function=style_function,
    marker=folium.CircleMarker()
).add_to(m)
m

### Now get the data into geopandas

In [94]:
import geopandas as gpd
import pandas as pd

In [137]:
# Get the data again
params = {'f': 'json', 'limit': 500}
response = requests.get('https://ogcapi.bgs.ac.uk/collections/recentearthquakes/items', params=params)
data = response.json()

In [138]:
earthquakes = gpd.GeoDataFrame.from_features(data['features'])
earthquakes = earthquakes.set_geometry('geometry')
earthquakes.head()

,geometry,earthquake_event_id,datetime,year,latitude,longitude,depth,ml,intensity,user_entered,date_entered,user_updated,date_updated
0,POINT (-3.70000 53.86000),86123.0,2017-04-05T15:51:03,2017,53.86,-3.70,2.8,1.0,0.0,APBE,2018-08-29T17:28:59,None,None
1,POINT (0.44000 51.88000),92221.0,1956-05-04T23:26:08,1956,51.88,0.44,NaN,3.0,0.0,APBE,2018-08-29T17:29:10,None,None
2,POINT (-3.12000 55.87000),92222.0,1967-11-28T04:07:36,1967,55.87,-3.12,NaN,1.8,0.0,APBE,2018-08-29T17:29:10,None,None
3,POINT (-2.97000 55.19000),92223.0,1968-03-08T05:26:48,1968,55.19,-2.97,NaN,3.2,0.0,APBE,2018-08-29T17:29:10,None,None
4,POINT (-3.06000 55.93000),92224.0,1968-10-16T17:02:40,1968,55.93,-3.06,NaN,3.1,0.0,APBE,2018-08-29T17:29:10,None,None


In [139]:
earthquakes.dtypes

geometry               geometry
earthquake_event_id     float64
datetime                 object
year                     object
latitude                float64
longitude               float64
depth                   float64
ml                      float64
intensity               float64
user_entered             object
date_entered             object
user_updated             object
date_updated             object
dtype: object

In [140]:
earthquakes.rename(columns={'earthquake_event_id': 'id', 'ml': 'magnitude'}, inplace=True)
# earthquakes['datetime'] = pd.to_datetime(earthquakes['datetime']) # leave as string for now
earthquakes['id'] = earthquakes['id'].astype(int)
earthquakes = earthquakes[['id', 'datetime', 'magnitude', 'intensity', 'depth', 'geometry']]
earthquakes.set_crs(epsg=4277, inplace=True)
earthquakes.head()

,id,datetime,magnitude,intensity,depth,geometry
0,86123,2017-04-05T15:51:03,1.0,0.0,2.8,POINT (-3.70000 53.86000)
1,92221,1956-05-04T23:26:08,3.0,0.0,NaN,POINT (0.44000 51.88000)
2,92222,1967-11-28T04:07:36,1.8,0.0,NaN,POINT (-3.12000 55.87000)
3,92223,1968-03-08T05:26:48,3.2,0.0,NaN,POINT (-2.97000 55.19000)
4,92224,1968-10-16T17:02:40,3.1,0.0,NaN,POINT (-3.06000 55.93000)


In [141]:
earthquakes.dtypes

id              int64
datetime       object
magnitude     float64
intensity     float64
depth         float64
geometry     geometry
dtype: object

In [142]:
earthquakes.describe()

,id,magnitude,intensity,depth
count,500.000000,500.000000,500.000000,224.000000
mean,92480.414000,1.732400,0.120000,3.837946
std,326.010964,0.744459,0.653034,4.490742
min,86123.000000,1.000000,0.000000,0.000000
25%,92359.500000,1.200000,0.000000,0.000000
50%,92494.500000,1.500000,0.000000,4.000000
75%,92631.250000,2.200000,0.000000,5.000000
max,92764.000000,4.800000,5.000000,34.600000


In [143]:
big_quakes = earthquakes[earthquakes['magnitude'] >= 4]
big_quakes

,id,datetime,magnitude,intensity,depth,geometry
44,92264,1970-08-09T20:09:01,4.1,5.0,20.9,POINT (-2.47000 54.50000)
52,92272,1971-03-23T20:05:18,4.7,0.0,NaN,POINT (2.80000 59.40000)
113,92336,1974-02-25T20:03:43,4.1,0.0,NaN,POINT (-3.12000 51.64000)
240,92485,1976-08-18T20:45:52,4.2,0.0,NaN,POINT (2.00000 62.00000)
289,92536,1977-04-06T19:32:04,4.4,0.0,NaN,POINT (3.00000 61.50000)
295,92542,1977-05-02T20:36:12,4.4,0.0,NaN,POINT (5.00000 59.00000)
324,92572,1977-07-26T03:48:32,4.8,0.0,NaN,POINT (3.50000 61.50000)
348,92600,1977-12-05T03:44:57,4.4,0.0,NaN,POINT (2.50000 61.50000)
416,92676,1978-09-19T14:52:38,4.7,0.0,8.0,POINT (1.75000 62.25000)


In [144]:
big_quakes_json = big_quakes.to_json()

In [160]:
# Define an initial map centred on and zoomed to the UK
m2 = folium.Map(location=[58, -2], zoom_start=5)

# Create popup
popup = folium.features.GeoJsonPopup(fields=["datetime", "magnitude", "intensity", "depth"])

# Populate and display the map
folium.GeoJson(big_quakes_json, popup=popup).add_to(m2)
m2